In [3]:
corpus_name = 'entities'

# load validation split
from beir.datasets.data_loader import GenericDataLoader

# e.g. to test entity retrieval
data_path = '/ivi/ilps/personal/svakule/spoken_qa/WD18/'
corpus_path = data_path + 'corpus/%s.jsonl' % corpus_name  # relations.jsonl
query_path = data_path + 'queries/original.jsonl'  # wav2vec2-base-960h.jsonl
qrels_path = data_path + 'qrels/valid_%s.tsv' % corpus_name  # valid_relations.tsv train_entities.tsv train_relations.tsv

corpus, queries, qrels = GenericDataLoader(corpus_file=corpus_path, 
                                           query_file=query_path, 
                                           qrels_file=qrels_path).load_custom()

In [4]:
from beir.retrieval.search.dense import DenseRetrievalExactSearch as DRES
from beir.retrieval.evaluation import EvaluateRetrieval
from beir.retrieval import models


model_name = "msmarco-distilbert-base-tas-b"

model = DRES(models.SentenceBERT(model_name))
retriever = EvaluateRetrieval(model, score_function="dot")
results1 = retriever.retrieve(corpus, queries)
ndcg, _map, recall, precision = retriever.evaluate(qrels, results1, retriever.k_values)
# acc = precision['P@1']
# print(acc)
print(ndcg, _map, recall, precision)

RuntimeError: CUDA error: all CUDA-capable devices are busy or unavailable